In [10]:
import pmagpy
import pmagpy.pmagplotlib as pmagplotlib
import pmagpy.ipmag as ipmag
import pmagpy.pmag as pmag
import pmagpy.contribution_builder as cb
from pmagpy import convert_2_magic as convert
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import vgptools
%matplotlib inline

In [11]:
home = Path(os.getcwd())
data_folder = home/'data'
magic_folder = home/'magic'

In [12]:
## Kosterov & Perrin (1996)
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 10/07/2024): 
#  
#
#  Location: samples.site
#  ID: samples.sample (had to add this column to remove ambiguities in site numbers)
#  Lat: samples.lat
#  Long: samples.lon
#  N: samples.dir_n_total_specimens
#  n: samples.dir_n_specimens
#  Inc: samples.dir_inc
#  Dec: samples.dir_dec
#  k: samples.dir_k
#  a95: samples.dir_alpha95
#  GroupID: samples.site ()
#  
# Assumptions:
#
#  samples.result_type = "i"
#  samples.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  samples.citations = "10.1016/0012-821X(96)00005-2"
#  samples.age = 180
#  samples.age_sigma = 5
#  samples.age_unit = "Ma"
#  samples.dir_tilt_correction = 0
#  samples.result_quality = "g" if group (col. N) is not nan, otherwise "b"

kosterov1996_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Kosterov 1996',skiprows=4).dropna(how='all')
#display(kosterov1996_data)

## Version used to generate dataframes currently on MagIC (original method)

In [20]:
def add_vgps(sites_df):
    # Outputs dataframe with columns: vgp_lat, vgp_lon, vgp_dp, vgp_dm
    data=sites_df[['dir_dec','dir_inc','dir_alpha95','lat','lon']].to_numpy()
    vgps=np.array(pmag.dia_vgp(data)) 
    vgps=vgps.transpose()
    return(pd.DataFrame(vgps,columns=['vgp_lon','vgp_lat','vgp_dp','vgp_dm'],index=sites_df.index))

def dataframe_average(dir_df, nb=500, column_map=None, flip=None, return_distribution=False):
    """
    Performs a bootstrap for direction DataFrame with parametric bootstrap,
    providing bootstrap kappa parameter

    Parameters
    ----------
    dir_df : Pandas DataFrame with columns:
        dir_dec : mean declination
        dir_inc : mean inclination
        dir_n : number of data points in mean
        dir_k : Fisher k statistic for mean
    nb : number of bootstraps, default is 500; if np = 0, do not bootstrap (instead, take Fisher average of input data)
    flip : whether/how to flip polarity of input directions
        None : Do not flip (assumes all magnetization vectors are normal) - Default
        'norm_nh' : flip so all directions are aligned with normal assuming northern hemisphere (N/down)
        'norm_sh' : flip so all directions are aligned with normal assuming southern hemisphere (S/down)
        'rev_nh' : flip so all directions are aligned with reversed assuming northern hemisphere (S/up)
        'rev_sh' : flip so all directions are aligned with reversed assuming southern hemisphere (N/up)
        'trim_norm_nh', 'trim_norm_sh', etc. : exclude directions that are >90 degrees normal/reversed in specified hemisphere
    return_distribution : return DataFrame of dec, inc, kappa values
            (default: False)

    Returns
    -------
    if return_distribution is False:
        boot_results: Pandas DataFrame with columns:
            dir_dec : bootstrapped median declination value
            dir_inc : bootstrapped median inclination value
            dir_k : bootstrapped median kappa value
            dir_n : number of rows in original dataframe
            dir_alpha95: A95 associated with median kappa
    if return_distribution is True:
        boot_results: Pandas DataFrame with all bootstrapped values of columns above
    
  
    """
    dir_df=dir_df.dropna()
    N = dir_df.shape[0] # Note: N counts only non-NaN elements
    if N>1:
        if column_map is not None:
            dir_df=dir_df.rename(columns=column_map)
        if flip is not None:
            dir_df=dataframe_flip(dir_df, how=flip)
        all_boot_results = pd.DataFrame(np.zeros((nb,5)),columns=['dir_dec','dir_inc','dir_n','dir_k','dir_alpha95'])
        if (nb>0):
            for k in tqdm(range(nb)):
                boot_di=dir_df.apply(lambda x: np.array(list(ipmag.fishrot(k=x['dir_k'],n=int(x['dir_n']),dec=x['dir_dec'],inc=x['dir_inc'],di_block=False))).T,axis=1).explode().apply(lambda x: pd.Series({'dir_dec':x[0],'dir_inc':x[1]}))
                all_boot_results.iloc[k,:] = pd.Series(pmag.dir_df_fisher_mean(boot_di)).drop(index=['csd','r']).rename(index={'alpha95':'dir_alpha95',
                                          'dec':'dir_dec',
                                          'inc':'dir_inc',
                                          'k':'dir_k',
                                          'n':'dir_n'})

            b = 20.**(1./(N -1.))-1.
            if return_distribution:
                results = all_boot_results
                results['a']=1-b*(N-1.)/((N*(results['dir_k']-1.))-1.)
                results['dir_n']=N
                results['dir_alpha95']=np.degrees(np.arccos(results['a']))
                results['result_type']='a'
            else:
                fisher_avg=pd.DataFrame([pmag.dir_df_fisher_mean(all_boot_results)]).rename(columns={'alpha95':'dir_alpha95',
                                          'dec':'dir_dec',
                                          'inc':'dir_inc',
                                          'k':'dir_k',
                                          'n':'dir_n'})
                fisher_avg['dir_k'] = np.median(all_boot_results['dir_k'].to_numpy())
                fisher_avg['dir_n'] = N
                a=1-b*(N-1.)/((N*(fisher_avg['dir_k']-1.))-1.)
                fisher_avg['dir_alpha95'] = np.degrees(np.arccos(a))
                results=fisher_avg[['dir_dec','dir_inc','dir_k','dir_n','dir_alpha95']]
                results['result_type']='a'
        else:
            fisher_avg=pd.DataFrame([pmag.dir_df_fisher_mean(dir_df)]).rename(columns={'alpha95':'dir_alpha95',
                                      'dec':'dir_dec',
                                      'inc':'dir_inc',
                                      'k':'dir_k',
                                      'n':'dir_n'})
            results=fisher_avg[['dir_dec','dir_inc','dir_k','dir_n','dir_alpha95']]
            results['result_type']='a'
    else:
        results=dir_df[['dir_dec','dir_inc','dir_k','dir_alpha95']]
        results['dir_n']=1
        results['result_type']='i'
    if column_map is not None:
        column_map_r=dict(zip(column_map.values(),column_map.keys()))
        results=results.rename(columns=column_map_r)
    return(results)

In [35]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
#sites_df['sample']=kosterov1996_data['ID'].astype('string')
#sites_df['site']=kosterov1996_data['GroupID'].astype('string')
#sites_df['site']=sites_df['site'].fillna('N/A')

sites_df['site']=kosterov1996_data['ID'].astype('string')
sites_df['site_alternatives']=kosterov1996_data['GroupID'].astype('string')
sites_df['site_alternatives']=sites_df['site_alternatives'].fillna('N/A')
sites_df['location']=kosterov1996_data['Location']
sites_df['lat']=kosterov1996_data['Lat']
sites_df['lon']=kosterov1996_data['Long']
sites_df['dir_n_total_specimens']=kosterov1996_data['N']
sites_df['dir_n_specimens']=kosterov1996_data['n']
sites_df['dir_dec']=kosterov1996_data['Dec']
sites_df['dir_inc']=kosterov1996_data['Inc']
sites_df['dir_alpha95']=kosterov1996_data['a95']
sites_df['dir_k']=kosterov1996_data['k']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1016/0012-821X(96)00005-2"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= 180
sites_df['age_sigma']= 5
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['location'].isna(),'result_quality']='b'
sites_df['elevation']=kosterov1996_data['Alt(m)']
sites_df['dir_tilt_correction']=0
#with pd.option_context('display.max_rows', 1000, 'display.max_columns', 50):
    #display(sites_df)

In [36]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)
sites_df=sites_df.round(2)

In [37]:
descr = [''] * len(sites_df['site'])
descr[53]='site SP-17 not used'
sites_df['description']=descr
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sites_df)

,site,site_alternatives,location,lat,lon,dir_n_total_specimens,dir_n_specimens,dir_dec,dir_inc,dir_alpha95,dir_k,lithologies,geologic_classes,result_type,method_codes,citations,geologic_types,age,age_sigma,age_unit,dir_tilt_correction,result_quality,elevation,vgp_lon,vgp_lat,vgp_dp,vgp_dm,description
0,M-101,GM-28,Mafika,-29.07,28.38,3.0,3.0,328.1,-74.9,3.0,1738.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3075.0,232.02,51.26,4.99,5.47,
1,M-100,GM-28,Mafika,-29.07,28.38,3.0,3.0,328.5,-75.8,2.0,3680.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3065.0,230.08,50.36,3.39,3.68,
2,M-95,GM-27,Mafika,-29.07,28.38,5.0,4.0,337.1,-58.3,2.9,1028.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3025.0,264.63,68.67,3.17,4.29,
3,M-93,GM-26,Mafika,-29.07,28.38,3.0,3.0,334.5,-54.0,3.1,1601.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3015.0,277.65,67.72,3.04,4.34,
4,M-91,GM-26,Mafika,-29.07,28.38,3.0,3.0,335.2,-54.5,2.4,2683.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3000.0,276.13,68.22,2.39,3.38,
5,M-89,GM-26,Mafika,-29.07,28.38,3.0,3.0,336.2,-54.1,2.4,2618.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2985.0,276.86,69.09,2.36,3.37,
6,M-88,GM-26,Mafika,-29.07,28.38,3.0,3.0,334.9,-54.8,2.0,3641.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2975.0,275.44,67.93,2.00,2.83,
7,M-86,GM-25,Mafika,-29.07,28.38,5.0,5.0,336.0,-49.9,1.3,3303.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2960.0,287.92,69.17,1.16,1.74,
8,M-84,GM-24,Mafika,-29.07,28.38,3.0,3.0,333.3,-54.0,1.9,4423.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2950.0,277.94,66.73,1.87,2.66,
9,M-82,GM-24,Mafika,-29.07,28.38,3.0,3.0,333.1,-52.8,2.9,1799.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2935.0,280.90,66.66,2.77,4.01,


In [26]:
# This block follows Kosterov and Perrin convention of combining flows into flow groups for averaging 
# This dataframe has been constructed to compare with their paper, though not uploaded to MagIC

sample_map={'dir_n_specimens':'dir_n'}
# Compile site groups dataframe
site_groups_df=pd.DataFrame([])
samples_group=sites_df.groupby('site_alternatives')
site_groups_df['groups']=samples_group.groups.keys()
site_groups_df['sites']=samples_group['site'].unique().apply(':'.join).reset_index(drop=True)#.drop('site',axis=1)
site_groups_df['site_type']='Outcrop'
site_groups_df['geologic_classes']=samples_group['geologic_classes'].unique().apply(':'.join).reset_index(drop=True)#.drop('site',axis=1)
site_groups_df['lithologies']='Basalt'
site_groups_df['location']=samples_group.first()['location'].reset_index(drop=True)
site_groups_df['lat_s']=samples_group['lat'].min().reset_index(drop=True)
site_groups_df['lat_n']=samples_group['lat'].max().reset_index(drop=True)
site_groups_df['lon_e']=samples_group['lon'].max().reset_index(drop=True)
site_groups_df['lon_w']=samples_group['lon'].min().reset_index(drop=True)
site_groups_df['age']=180
site_groups_df['age_sigma']=5
site_groups_df['age_unit']="Ma"
site_groups_df['dir_n_sites']=samples_group.size()

# add df of site group means here, merge
site_group_avgs_df=pd.DataFrame([])
site_group_avgs_df['groups']=kosterov1996_data['GroupID'].astype('string')
site_group_avgs_df['dir_dec']=kosterov1996_data['Dec.1']
site_group_avgs_df['dir_inc']=kosterov1996_data['Inc.1']
site_group_avgs_df['dir_alpha95']=kosterov1996_data['a95.1']
site_group_avgs_df['dir_k']=kosterov1996_data['k.1']
site_group_avgs_df['vgp_lat']=kosterov1996_data['VGP Lat']
site_group_avgs_df['vgp_lon']=kosterov1996_data['VGP Long']
site_group_avgs_df=site_group_avgs_df[site_group_avgs_df['dir_dec'].notna()]
site_groups_df = pd.merge(site_groups_df,site_group_avgs_df,how='inner',on='groups')

# count dir_n_sites for given site group and add to df 
sites_count = np.zeros(len(site_groups_df.sites))
char = ":"

for i in range(len(site_groups_df.sites)):
    string = site_groups_df.sites[i]
    sites_count[i] = string.count(char) +1
site_groups_df.dir_n_sites = sites_count

    
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(site_groups_df)

In [27]:
location_map={'dir_n_sites':'dir_n'}
# Compile locations dataframe
locations_df=pd.DataFrame([])
sites_group=site_groups_df.groupby('location')
locations_df['location']=sites_group.groups.keys()
locations_df['sites']=sites_group['sites'].unique().apply(':'.join).reset_index(drop=True)
locations_df['location_type']='Outcrop'
locations_df['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index(drop=True)#.drop('location',axis=1)
locations_df['lithologies']='Basalt'
#locations_df['description']=sites_group.first()['description'].reset_index(drop=True)
locations_df['lat_s']=sites_group['lat_s'].min().reset_index(drop=True)
locations_df['lat_n']=sites_group['lat_n'].max().reset_index(drop=True)
locations_df['lon_e']=sites_group['lon_e'].max().reset_index(drop=True)
locations_df['lon_w']=sites_group['lon_w'].min().reset_index(drop=True)
locations_df['age']=180
locations_df['age_sigma']=5
locations_df['age_unit']="Ma"

sites_count = np.zeros(len(locations_df.sites))
char = ":"

for i in range(len(locations_df.sites)):
    string = locations_df.sites[i]
    sites_count[i] = string.count(char) + 1
locations_df['dir_n_sites'] = sites_count

In [28]:
locations_df=pd.concat([locations_df,sites_group.apply(lambda x: dataframe_average(x,nb=0,column_map=sample_map)).reset_index()],axis=1)
# Need to rename columns: now our "n" is numbers of sites, since locations_df is the locations dataframe (a site-level average).
locations_df=locations_df.rename({'dir_n_samples':'dir_n_sites'})
locations_df=locations_df.round(1)

In [29]:
del locations_df['level_1']
del locations_df['dir_n_specimens']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(locations_df)

,location,sites,location_type,geologic_classes,lithologies,lat_s,lat_n,lon_e,lon_w,age,age_sigma,age_unit,dir_n_sites,location,dir_dec,dir_inc,dir_k,dir_alpha95,result_type
0,Mafika,M-1:M-35:M-33:M-37:M-39:M-41:M-43:M-45:M-47:M-...,Outcrop,Igneous:Extrusive,Basalt,-29.1,-29.1,28.4,28.4,180,5,Ma,47.0,Mafika,340.2,-55.1,49.2,3.9,a
1,Nazareth,N-1:N-2:N-3:N-4:N-5,Outcrop,Igneous:Extrusive,Basalt,-29.4,-29.4,27.8,27.8,180,5,Ma,5.0,Nazareth,161.9,64.3,82.0,10.2,a
2,Rhodes,R-1:R-6:R-2:R-3,Outcrop,Igneous:Extrusive,Basalt,-30.8,-30.8,28.0,28.0,180,5,Ma,4.0,Rhodes,153.0,44.5,89.6,13.1,a
3,Sani Pass,SP-1:SP-20:SP-19:SP-21a:SP-21:SP-22:SP-4:SP-2:...,Outcrop,Igneous:Extrusive,Basalt,-29.6,-29.6,29.3,29.3,180,5,Ma,16.0,Sani Pass,156.6,49.2,34.1,7.5,a


In [30]:
contribution_df=pd.DataFrame({'reference':["https://doi.org/10.1016/0012-821X(96)00005-2"],'lab_names':["Laboratoire de Geophysique et Tectonique, Monpellier University"]},index=[0])
print(contribution_df)

# Changing dataframes to dictionaries for export
contribution_dicts=contribution_df.fillna('').to_dict('records')
locations_dicts=locations_df.fillna('').to_dict('records')
sites_dicts = sites_df.fillna('').to_dict('records')

                                      reference  \
0  https://doi.org/10.1016/0012-821X(96)00005-2   

                                           lab_names  
0  Laboratoire de Geophysique et Tectonique, Monp...  


In [31]:
pmag.magic_write(str(magic_folder/'kosterov1996_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'kosterov1996_sites.txt', sites_dicts, 'sites')
pmag.magic_write(magic_folder/'kosterov1996_contribution.txt', contribution_dicts, 'contribution')

4  records written to file  /Users/rkepler/codes/published-data-to-magic-main1/magic/kosterov1996_locations.txt
73  records written to file  /Users/rkepler/codes/published-data-to-magic-main1/magic/kosterov1996_sites.txt
1  records written to file  /Users/rkepler/codes/published-data-to-magic-main1/magic/kosterov1996_contribution.txt


(True,
 PosixPath('/Users/rkepler/codes/published-data-to-magic-main1/magic/kosterov1996_contribution.txt'))

## Cleaned up/updated version 

In [38]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
#sites_df['sample']=kosterov1996_data['ID'].astype('string')
#sites_df['site']=kosterov1996_data['GroupID'].astype('string')
#sites_df['site']=sites_df['site'].fillna('N/A')

sites_df['site']=kosterov1996_data['ID'].astype('string')
sites_df['site_alternatives']=kosterov1996_data['GroupID'].astype('string')
sites_df['site_alternatives']=sites_df['site_alternatives'].fillna('N/A')
sites_df['location']=kosterov1996_data['Location']
sites_df['lat']=kosterov1996_data['Lat']
sites_df['lon']=kosterov1996_data['Long']
sites_df['dir_n_total_specimens']=kosterov1996_data['N']
sites_df['dir_n_specimens']=kosterov1996_data['n']
sites_df['dir_dec']=kosterov1996_data['Dec']
sites_df['dir_inc']=kosterov1996_data['Inc']
sites_df['dir_alpha95']=kosterov1996_data['a95']
sites_df['dir_k']=kosterov1996_data['k']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1016/0012-821X(96)00005-2"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= 180
sites_df['age_sigma']= 5
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['location'].isna(),'result_quality']='b'
sites_df['elevation']=kosterov1996_data['Alt(m)']
sites_df['dir_tilt_correction']=0
#with pd.option_context('display.max_rows', 1000, 'display.max_columns', 50):
    #display(sites_df)

In [39]:
# Calculate and attach VGPs
sites_df=vgptools.calculate_vgps(sites_df)

In [40]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sites_df)

,site,site_alternatives,location,lat,lon,dir_n_total_specimens,dir_n_specimens,dir_dec,dir_inc,dir_alpha95,dir_k,lithologies,geologic_classes,result_type,method_codes,citations,geologic_types,age,age_sigma,age_unit,dir_tilt_correction,result_quality,elevation,vgp_lon,vgp_lat,vgp_dp,vgp_dm
0,M-101,GM-28,Mafika,-29.07,28.38,3.0,3.0,328.1,-74.9,3.0,1738.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3075.0,232.0,51.3,5.0,5.5
1,M-100,GM-28,Mafika,-29.07,28.38,3.0,3.0,328.5,-75.8,2.0,3680.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3065.0,230.1,50.4,3.4,3.7
2,M-95,GM-27,Mafika,-29.07,28.38,5.0,4.0,337.1,-58.3,2.9,1028.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3025.0,264.6,68.7,3.2,4.3
3,M-93,GM-26,Mafika,-29.07,28.38,3.0,3.0,334.5,-54.0,3.1,1601.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3015.0,277.6,67.7,3.0,4.3
4,M-91,GM-26,Mafika,-29.07,28.38,3.0,3.0,335.2,-54.5,2.4,2683.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,3000.0,276.1,68.2,2.4,3.4
5,M-89,GM-26,Mafika,-29.07,28.38,3.0,3.0,336.2,-54.1,2.4,2618.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2985.0,276.9,69.1,2.4,3.4
6,M-88,GM-26,Mafika,-29.07,28.38,3.0,3.0,334.9,-54.8,2.0,3641.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2975.0,275.4,67.9,2.0,2.8
7,M-86,GM-25,Mafika,-29.07,28.38,5.0,5.0,336.0,-49.9,1.3,3303.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2960.0,287.9,69.2,1.2,1.7
8,M-84,GM-24,Mafika,-29.07,28.38,3.0,3.0,333.3,-54.0,1.9,4423.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2950.0,277.9,66.7,1.9,2.7
9,M-82,GM-24,Mafika,-29.07,28.38,3.0,3.0,333.1,-52.8,2.9,1799.0,Basalt,Igneous:Extrusive,i,DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T,10.1016/0012-821X(96)00005-2,Lava Flow,180,5,Ma,0,g,2935.0,280.9,66.7,2.8,4.0


In [41]:
locations_df1=vgptools.roll_up_sites_locations(sites_df,location_parameter='location',nb=0)


In [42]:
locations_df1

,location,sites,lithologies,lat_s,lat_n,lon_e,lon_w,dir_dec,dir_inc,dir_k,dir_n_samples,dir_alpha95,result_type
0,Mafika,M-101:M-100:M-95:M-93:M-91:M-89:M-88:M-86:M-84...,Basalt,-29.1,-29.1,28.4,28.4,158.9,54.7,64.7,47.0,2.6,a
1,Nazareth,N-1:N-2:N-3:N-4:N-5,Basalt,-29.4,-29.4,27.8,27.8,156.5,49.2,36.8,16.0,6.2,a
2,Rhodes,R-1:R-6:R-2:R-3,Basalt,-30.8,-30.8,28.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Sani Pass,SP-22:SP-21a:SP-21:SP-20:SP-19:SP-18:SP-17:SP-...,Basalt,-29.6,-29.6,29.3,29.3,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
vgptools.roll_up_sites_locations??